In [1]:
%run 2-0-det2-training-common.ipynb

Sun Feb 11 01:20:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-PCIE-32GB           Off | 00000000:81:00.0 Off |                    0 |
| N/A   45C    P0              40W / 250W |   1416MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# faster-rcnn

# Import necessary modules and classes

In [2]:
#from .detectron2.tools.train_net import Trainer  # Uncomment if this is required
#from detectron2.engine import DefaultTrainer  # Uncomment if this is required
# select from modelzoo here: https://github.com/facebookresearch/detectron2/blob/master/MODEL_ZOO.md#coco-object-detection-baselines
from detectron2.config import get_cfg
#from detectron2.evaluation.coco_evaluation import COCOEvaluator  # Uncomment if this is required
import os

# Create a configuration object
cfg = get_cfg()
cfg.OUTPUT_DIR = "output/faster-rcnn"

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")

# cfg.MODEL.RETINANET.NUM_CLASSES=
cfg.MODEL.RETINANET.NUM_CLASSES=4
cfg.MODEL.ROI_HEADS.NUM_CLASSES=4
cfg.MODEL.ROI_BOX_HEAD.FED_LOSS_NUM_CLASSES=4
cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES=4

# Set training and validation datasets
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)

# Set number of data loading workers
cfg.DATALOADER.NUM_WORKERS = 3

# Set batch size and base learning rate
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.05

# Set warm-up iterations, maximum iterations, steps, and gamma for the learning rate scheduler
cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.MAX_ITER = 30000  # Adjust up if val mAP is still rising, adjust down if overfit
cfg.SOLVER.STEPS = (1000, 1500)
# cfg.SOLVER.GAMMA = 0.05

# Set batch size per image and number of classes for ROI heads
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64


# Set evaluation period during training
cfg.TEST.EVAL_PERIOD = 500

# Create output directory if it does not exist
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)



# Create a CocoTrainer object
trainer = CocoTrainer(cfg)

if cfg.TEST.AUG.ENABLED:
  trainer.register_hooks(
    [hooks.EvalHook(0, lambda: trainer.test_with_TTA(cfg, trainer.model))]   #this block uses a hook to run evalutaion periodically
  )                                           #https://detectron2.readthedocs.io/en/latest/modules/engine.html#detectron2.engine.hooks.EvalHook


# Resume training or load from a checkpoint
trainer.resume_or_load(resume=True)
# trainer.resume_or_load(resume=False)

[02/11 01:20:45 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

In [3]:
# trainer.train()

In [4]:
#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.15
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.6 # iou

predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_test", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "my_dataset_test")
inference_on_dataset(trainer.model, val_loader, evaluator)

[02/11 01:21:03 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from output/faster-rcnn/model_final.pth ...
WARNING [02/11 01:21:05 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[02/11 01:21:06 d2.data.datasets.coco]: Loaded 67 images in COCO format from /work/van-speech-nlp/jindaznb/j-vis/Forestfire2024-1/test/_annotations.coco.json
[02/11 01:21:06 d2.data.build]: Distribution of instances among all 4 categories:
|  category  | #instances   |   category    | #instances   |  category  | #instances   |
|:----------:|:-------------|:-------------:|:-------------|:----------:|:-------------|
| Alive Tree | 6961         | Beetle-Fire.. | 752          | Dead Tree  | 531          |
|   Debris   | 512          |               |              |            |              |
|   total    | 8756         |               |              |            |              |
[02/11 01:21:06 

/work/van-speech-nlp/det2env/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[02/11 01:21:15 d2.evaluation.evaluator]: Inference done 1/67. Dataloading: 0.9913 s/iter. Inference: 7.9302 s/iter. Eval: 0.0034 s/iter. Total: 8.9339 s/iter. ETA=0:09:49
[02/11 01:21:20 d2.evaluation.evaluator]: Inference done 19/67. Dataloading: 0.0059 s/iter. Inference: 0.2697 s/iter. Eval: 0.0024 s/iter. Total: 0.2782 s/iter. ETA=0:00:13
[02/11 01:21:25 d2.evaluation.evaluator]: Inference done 35/67. Dataloading: 0.0062 s/iter. Inference: 0.2662 s/iter. Eval: 0.0300 s/iter. Total: 0.3027 s/iter. ETA=0:00:09
[02/11 01:21:30 d2.evaluation.evaluator]: Inference done 54/67. Dataloading: 0.0064 s/iter. Inference: 0.2656 s/iter. Eval: 0.0193 s/iter. Total: 0.2916 s/iter. ETA=0:00:03
[02/11 01:21:34 d2.evaluation.evaluator]: Total inference time: 0:00:17.993874 (0.290224 s / iter per device, on 1 devices)
[02/11 01:21:34 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:16 (0.264920 s / iter per device, on 1 devices)
[02/11 01:21:34 d2.evaluation.coco_evaluation]: FUCK YO

OrderedDict([('bbox',
              {'AP': 29.109772544149987,
               'AP50': 47.135814926581034,
               'AP75': 31.915430971444046,
               'APs': 8.206945133051516,
               'APm': 33.88223926763407,
               'APl': 69.00413218541402,
               'AP-Alive Tree': 34.21851697564814,
               'AP-Beetle-Fire Tree': 20.343100256808373,
               'AP-Dead Tree': 29.384651606934753,
               'AP-Debris': 32.49282133720867})])

In [5]:
%ls ./output/

coco_instances_results.json  instances_predictions.pth
faster-rcnn/                 retina_fpn/


In [6]:
# cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
# cfg.DATASETS.TEST = ("my_dataset_test", )
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
# predictor = DefaultPredictor(cfg)
# test_metadata = MetadataCatalog.get("my_dataset_test")

In [7]:
# from detectron2.utils.visualizer import ColorMode
# import glob

# for imageName in glob.glob('/content/test/*jpg'):
#   im = cv2.imread(imageName)
#   outputs = predictor(im)
#   v = Visualizer(im[:, :, ::-1],
#                 metadata=test_metadata, 
#                 scale=0.8
#                  )
#   out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
#   cv2_imshow(out.get_image()[:, :, ::-1])
